# unsafe.Pointer usage survey



In [93]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import json
from datetime import datetime

## Analyse von Match Daten

In [71]:
# rg unsafe.Pointer --context 5 --json zsyscall_freebsd_386.go

testdata = r"""[{"type":"begin","data":{"path":{"text":"zsyscall_freebsd_386.go"}}},
{"type":"context","data":{"path":{"text":"zsyscall_freebsd_386.go"},"lines":{"text":"var _ syscall.Errno\n"},"line_number":13,"absolute_offset":242,"submatches":[]}},
{"type":"context","data":{"path":{"text":"zsyscall_freebsd_386.go"},"lines":{"text":"\n"},"line_number":14,"absolute_offset":262,"submatches":[]}},
{"type":"context","data":{"path":{"text":"zsyscall_freebsd_386.go"},"lines":{"text":"// THIS FILE IS GENERATED BY THE COMMAND AT THE TOP; DO NOT EDIT\n"},"line_number":15,"absolute_offset":263,"submatches":[]}},
{"type":"context","data":{"path":{"text":"zsyscall_freebsd_386.go"},"lines":{"text":"\n"},"line_number":16,"absolute_offset":328,"submatches":[]}},
{"type":"context","data":{"path":{"text":"zsyscall_freebsd_386.go"},"lines":{"text":"func getgroups(ngid int, gid *_Gid_t) (n int, err error) {\n"},"line_number":17,"absolute_offset":329,"submatches":[]}},
{"type":"match","data":{"path":{"text":"zsyscall_freebsd_386.go"},"lines":{"text":"\tr0, _, e1 := RawSyscall(SYS_GETGROUPS, uintptr(ngid), uintptr(unsafe.Pointer(gid)), 0)\n"},"line_number":18,"absolute_offset":388,"submatches":[{"match":{"text":"unsafe.Pointer"},"start":63,"end":77}]}},
{"type":"context","data":{"path":{"text":"zsyscall_freebsd_386.go"},"lines":{"text":"\tn = int(r0)\n"},"line_number":19,"absolute_offset":476,"submatches":[]}},
{"type":"context","data":{"path":{"text":"zsyscall_freebsd_386.go"},"lines":{"text":"\tif e1 != 0 {\n"},"line_number":20,"absolute_offset":489,"submatches":[]}},
{"type":"context","data":{"path":{"text":"zsyscall_freebsd_386.go"},"lines":{"text":"\t\terr = errnoErr(e1)\n"},"line_number":21,"absolute_offset":503,"submatches":[]}},
{"type":"context","data":{"path":{"text":"zsyscall_freebsd_386.go"},"lines":{"text":"\t}\n"},"line_number":22,"absolute_offset":524,"submatches":[]}},
{"type":"context","data":{"path":{"text":"zsyscall_freebsd_386.go"},"lines":{"text":"\treturn\n"},"line_number":23,"absolute_offset":527,"submatches":[]}},
{"type":"context","data":{"path":{"text":"zsyscall_freebsd_386.go"},"lines":{"text":"}\n"},"line_number":24,"absolute_offset":535,"submatches":[]}},
{"type":"context","data":{"path":{"text":"zsyscall_freebsd_386.go"},"lines":{"text":"\n"},"line_number":25,"absolute_offset":537,"submatches":[]}},
{"type":"context","data":{"path":{"text":"zsyscall_freebsd_386.go"},"lines":{"text":"// THIS FILE IS GENERATED BY THE COMMAND AT THE TOP; DO NOT EDIT\n"},"line_number":26,"absolute_offset":538,"submatches":[]}},
{"type":"context","data":{"path":{"text":"zsyscall_freebsd_386.go"},"lines":{"text":"\n"},"line_number":27,"absolute_offset":603,"submatches":[]}},
{"type":"context","data":{"path":{"text":"zsyscall_freebsd_386.go"},"lines":{"text":"func setgroups(ngid int, gid *_Gid_t) (err error) {\n"},"line_number":28,"absolute_offset":604,"submatches":[]}},
{"type":"match","data":{"path":{"text":"zsyscall_freebsd_386.go"},"lines":{"text":"\t_, _, e1 := RawSyscall(SYS_SETGROUPS, uintptr(ngid), uintptr(unsafe.Pointer(gid)), 0)\n"},"line_number":29,"absolute_offset":656,"submatches":[{"match":{"text":"unsafe.Pointer"},"start":62,"end":76}]}},
{"type":"context","data":{"path":{"text":"zsyscall_freebsd_386.go"},"lines":{"text":"\tif e1 != 0 {\n"},"line_number":30,"absolute_offset":743,"submatches":[]}},
{"type":"context","data":{"path":{"text":"zsyscall_freebsd_386.go"},"lines":{"text":"\t\terr = errnoErr(e1)\n"},"line_number":31,"absolute_offset":757,"submatches":[]}},
{"type":"context","data":{"path":{"text":"zsyscall_freebsd_386.go"},"lines":{"text":"\t}\n"},"line_number":32,"absolute_offset":778,"submatches":[]}},
{"type":"context","data":{"path":{"text":"zsyscall_freebsd_386.go"},"lines":{"text":"\treturn\n"},"line_number":33,"absolute_offset":781,"submatches":[]}},
{"type":"context","data":{"path":{"text":"zsyscall_freebsd_386.go"},"lines":{"text":"}\n"},"line_number":34,"absolute_offset":789,"submatches":[]}},
{"type":"context","data":{"path":{"text":"zsyscall_freebsd_386.go"},"lines":{"text":"\n"},"line_number":35,"absolute_offset":791,"submatches":[]}},
{"type":"context","data":{"path":{"text":"zsyscall_freebsd_386.go"},"lines":{"text":"// THIS FILE IS GENERATED BY THE COMMAND AT THE TOP; DO NOT EDIT\n"},"line_number":36,"absolute_offset":792,"submatches":[]}},
{"type":"context","data":{"path":{"text":"zsyscall_freebsd_386.go"},"lines":{"text":"\n"},"line_number":37,"absolute_offset":857,"submatches":[]}}
]
"""

td = json.loads(testdata)

In [61]:
matches = []

for i, line in enumerate(td):
    if line['type'] == 'match':
        context_lines = td[max(0, i-5) : min(len(td), i+1+5)]
        context = "".join([cl['data']['lines']['text'] for cl in context_lines])
        
        matches.append({
            'match': line['data']['lines']['text'],
            'context': context,
            'file': line['data']['path']['text'],
            'line_number': line['data']['line_number'],
            'absolute_offset': line['data']['absolute_offset'],
        })
        
df = pd.DataFrame(matches)

In [70]:
print(matches[1]['context'])

}

// THIS FILE IS GENERATED BY THE COMMAND AT THE TOP; DO NOT EDIT

func setgroups(ngid int, gid *_Gid_t) (err error) {
	_, _, e1 := RawSyscall(SYS_SETGROUPS, uintptr(ngid), uintptr(unsafe.Pointer(gid)), 0)
	if e1 != 0 {
		err = errnoErr(e1)
	}
	return
}



## Analyse von Projektdaten

In [94]:
dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S %z %Z')

projects_df = pd.read_csv('/home/johannes/studium/s14/masterarbeit/code/data-survey/data/projects.csv',
                         parse_dates=['created_at', 'last_pushed_at', 'updated_at'],
                         date_parser=dateparse)

In [97]:
projects_df[:5]

,rank,name,github_clone_url,number_of_stars,number_of_forks,github_id,created_at,last_pushed_at,updated_at,size,checkout_path
0,1,golang/go,https://github.com/golang/go.git,71166,10162,23096959,2014-08-19 04:33:40+00:00,2020-04-13 10:27:46+00:00,2020-04-13 11:24:54+00:00,209324,/home/johannes/studium/s14/masterarbeit/downlo...
1,2,kubernetes/kubernetes,https://github.com/kubernetes/kubernetes.git,64968,23145,20580498,2014-06-06 22:56:04+00:00,2020-04-13 11:08:13+00:00,2020-04-13 11:17:13+00:00,909251,/home/johannes/studium/s14/masterarbeit/downlo...
2,3,moby/moby,https://github.com/moby/moby.git,56803,16416,7691631,2013-01-18 18:10:57+00:00,2020-04-12 15:23:23+00:00,2020-04-13 08:34:49+00:00,152827,/home/johannes/studium/s14/masterarbeit/downlo...
3,4,avelino/awesome-go,https://github.com/avelino/awesome-go.git,53410,7088,21540759,2014-07-06 13:42:15+00:00,2020-04-13 09:42:24+00:00,2020-04-13 10:31:16+00:00,8027,/home/johannes/studium/s14/masterarbeit/downlo...
4,5,gohugoio/hugo,https://github.com/gohugoio/hugo.git,43139,4900,11180687,2013-07-04 15:26:26+00:00,2020-04-12 19:12:47+00:00,2020-04-13 10:14:32+00:00,86013,/home/johannes/studium/s14/masterarbeit/downlo...
